In [128]:
from z3 import *
from utils import *
from math import ceil
import numpy as np

### Instances

In [129]:
import os

def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Read m and n
    m = int(lines[0].strip())
    n = int(lines[1].strip())
    
    # Read l vector
    l = list(map(int, lines[2].strip().split()))
    
    # Read s vector
    s = list(map(int, lines[3].strip().split()))
    
    # Read D matrix
    D = []
    for line in lines[4:]:
        D.append(list(map(int, line.strip().split())))
    
    return {
        'm': m,
        'n': n,
        'l': l,
        's': s,
        'D': D
    }

def read_all_dat_files(directory):
    instances = []
    for filename in os.listdir(directory):
        if filename.endswith('.dat'):
            file_path = os.path.join(directory, filename)
            instance = read_dat_file(file_path)
            instances.append(instance)
    return instances

# Directory containing .dat files
directory = 'instances'

# Read all .dat files and populate instances
instances = read_all_dat_files(directory)

In [130]:
for i in instances:
    max_element = [max(max(row) for row in i['D'])]

max_d = max(max_element)
print(max_d)

339


# MCP

## Metodo 1

In [131]:
from z3 import *

def MCP_a(instance, timeout=5000):
    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']  # Distance matrix

    # Variables
    x = [[Bool(f'x_{i+1}_{j+1}') for j in range(n)] for i in range(m)]  # Adjust item names to start from 1
    y = [Int(f'y_{i+1}') for i in range(m)]  # Total size of items assigned to courier i

    # Initialize solver
    solver = Solver()

    # Constraint 1: Each item must be assigned to exactly one courier
    for j in range(n):
        item_assigned = [x[i][j] for i in range(m)]
        solver.add(exactly_one_np(item_assigned, f"item_{j+1}"))  # Adjust item names to start from 1

    # Constraint 2: The total size of items assigned to any courier must not exceed its load capacity
    for i in range(m):
        courier_assigned_size = 0
        for j in range(n):
            courier_assigned_size += If(x[i][j], s[j], 0)
        solver.add(y[i] == courier_assigned_size)
        solver.add(y[i] <= l[i])

    # Constraint 3: Each courier must pick at least one item
    for i in range(m):
        at_least_one_item = Or([x[i][j] for j in range(n)])
        solver.add(at_least_one_item)

    # Constraint 4: Each courier's path must start and end at the depot (location n+1)
    for i in range(m):
        path_starts_at_depot = Or([And(x[i][j], j == n) for j in range(n)])
        path_ends_at_depot = Or([And(x[i][j], j == n) for j in range(n)])
        solver.add(path_starts_at_depot)
        solver.add(path_ends_at_depot)

    # Define the maximum distance variable
    max_distance = Int('max_distance')

    # Distance constraints adjusted for depot start and end
    for i in range(m):
        courier_distance_expr = 0
        for j in range(n):
            courier_distance_expr += D[j][j+1] * If(x[i][j], 1, 0)  # Distance from j to j+1 for items assigned to courier i
        solver.add(max_distance >= courier_distance_expr)

    # Calculate upper_bound without using built-in sum
    upper_bound = 0
    for capacity in l:
        upper_bound += capacity

    # Binary search for minimizing max_distance
    lower_bound = 0

    while lower_bound < upper_bound:
        mid = (lower_bound + upper_bound) // 2
        solver.push()
        solver.add(max_distance <= mid)
        if solver.check() == sat:
            upper_bound = mid
        else:
            lower_bound = mid + 1
        solver.pop()

    # At the end, lower_bound will contain the minimum max_distance
    min_max_distance = lower_bound

    # Add the final constraint to get the actual model
    solver.add(max_distance == min_max_distance)

    # Check for satisfiability
    if solver.check() == sat:
        model = solver.model()
        print("Solution found:")
        for i in range(m):
            assigned_items = [j+1 for j in range(n) if is_true(model[x[i][j]])]  # Adjust item names to start from 1
            print(f"Courier {i+1} assigned items: {assigned_items}")  # Adjust courier names to start from 1
        print(f"Minimum maximum distance: {model[max_distance]}")
    else:
        print("No solution found")

## Metodo 2

In [132]:
def MCP(instance, timeout=5000):
    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']
    
    # Path variables: c courier went from node j to k
    path = [[[Bool(f"p_{c}_{i}_{j}") for j in range(n+1)] for i in range(n+1)] for c in range(m)]

    # Max distance in the distance matrix
    max_d = max(max(row) for row in D)

    # Calculate upper bound without using sum
    upperbound = ceil(n / m) * max_d

    def solve(instance, m, n, l, s, D, path, timeout):
        solver = Solver()
        solver.set(timeout=timeout)  # Set timeout

        # All couriers start from depot (node n) and end at depot
        for c in range(m):
            solver.add(exactly_one_seq([path[c][n][j] for j in range(n+1)], f"start_{c}"))
            solver.add(exactly_one_seq([path[c][i][n] for i in range(n+1)], f"end_{c}"))
        
        # Each item is taken by at most one courier
        for i in range(n):
            for j in range(n+1):
                solver.add(at_most_one_seq([path[c][i][j] for c in range(m)], f"item_{i}_{j}"))
        '''
        # Chain of couriers
        for c in range(m):
            for i in range(n+1):
                for j in range(n+1):
                    solver.add(Implies(path[c][i][j], exactly_one_seq([path[c][j][k] for k in range(n+1)], f"chain_{c}_{i}_{j}")))

        # Maximum load constraint
        for c in range(m):
            weight_set = []
            for i in range(n):
                for j in range(n+1):
                    weight_set.extend([path[c][i][j]] * s[i])
            solver.add(at_most_k_seq(weight_set, l[c]))
        '''

        # Ensure all items are picked up
        for i in range(n):
            solver.add(exactly_one_seq([path[c][i][j] for c in range(m) for j in range(n+1)], f"all_items_{i}"))

        # Ensure each courier picks at least one item
        for c in range(m):
            solver.add(at_least_one_seq([path[c][i][j] for i in range(n) for j in range(n+1)]))

        # Implicit constraints based on distances
        distances = [[] for _ in range(m)]
        for c in range(m):
            for i in range(n+1):
                for j in range(n+1):
                    distances[c].extend([path[c][i][j]] * D[i][j])
        '''
        lengths = [sum(is_true(dist) for dist in array) for array in distances]
        max_length = max(lengths)
        solver.add(max_length <= upperbound)
        '''
        # Check for satisfiability
        if solver.check() == sat:
            model = solver.model()
            return True, model, [(c, i, j) for c in range(m) for i in range(n+1) for j in range(n+1) if is_true(model[path[c][i][j]])]
        else:
            return False, None, []

    # Call solve function to check for any solution
    is_sat, model, solution = solve(instance, m, n, l, s, D, path, timeout)

    if is_sat:
        print("Found a solution:")
        for c in range(m):
            assigned_items = []
            for j in range(n+1):
                if any(is_true(model[path[c][i][j]]) for i in range(n+1)):
                    assigned_items.append(j)
            print(f"Courier {c}: {assigned_items}")

        # Print distances
        for c in range(m):
            distance = 0
            for i in range(n+1):
                for j in range(n+1):
                    if is_true(model[path[c][i][j]]):
                        distance += D[i][j]
            print(f"Distance for Courier {c}: {distance}")

    else:
        print("No solution found.")

    return solution


### Work on this below

In [153]:
from z3 import *
from math import ceil

def MCP(instance, timeout=5000):
    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']

    # Path variables: c courier went from node i to j
    path = [[[Bool(f"p_{c}_{i}_{j}") for j in range(n+2)] for i in range(n+2)] for c in range(m)]

    max_load = 100

    # Define maximum possible distance
    max_d = max([max(row) for row in D])
    upperbound = ceil(n/m) * max_d * 2

    def solve(m, n, l, s, D, path, timeout):
        solver = Solver()
        solver.set("timeout", timeout)  # Set timeout in milliseconds

        # All couriers start from depot (node n+1) and end at depot
        for c in range(m):
            solver.add(exactly_one_seq([path[c][n+1][j] for j in range(n+2)], f"start_{c}"))  # Start from depot (node n+1)
            solver.add(exactly_one_seq([path[c][i][n+1] for i in range(n+2)], f"end_{c}"))    # End at depot (node n+1)

        # Ensure each item is picked up exactly once
        for i in range(n+1):
            items_picked_up = [Or([path[c][i][j] for j in range(n+2)]) for c in range(m)]
            solver.add(exactly_one_seq(items_picked_up, f"item_{i}"))

        # Ensure each courier picks at least one item
        for c in range(m):
            solver.add(at_least_one_seq([path[c][i][j] for i in range(n+1) for j in range(n+1)]))

        # Constraints based on distances
        distances = [[] for _ in range(m)]
        for c in range(m):
            for i in range(n+1):  # Iterate up to n (since D is indexed from 0 to n)
                for j in range(n+1):
                    distances[c].extend([path[c][i][j]] * D[i][j])  # Adjust indices for D
        
        dist_per_courier = []
        for c in range(m):
            dist = sum([D[i][j] for i in range(n+1) for j in range(n+1) if is_true(solver.model()[path[c][i][j]])])
            dist_per_courier.append(dist)
            
        # max distance among all the couriers
        if dist_per_courier:
            max_dist = max(dist_per_courier)
        else:
            max_dist = 0  # If no solution found, distance is 0 or can be set to a meaningful default
        
        # Convert integers to lists of BoolRefs
        max_dist_bits = convert(max_dist, 4)  # Convert max_dist to 4-bit BoolRefs
        upperbound_bits = convert(upperbound, 7)  # Convert upperbound to 7-bit BoolRefs

        # Example of using a condition based on max_dist and upperbound
        P = Bool('P')  # Create a new Boolean variable
        is_pos = is_positive_difference(max_dist_bits, upperbound_bits)

        solver.add(Implies(is_pos, P))

        # Check for satisfiability
        check_result = solver.check()
        if check_result == sat:
            model = solver.model()
            return True, model, [(c, i, j) for c in range(m) for i in range(n+2) for j in range(n+2) if is_true(model[path[c][i][j]])]
        elif check_result == unsat:
            return False, None, []
        else:
            raise RuntimeError("Solver could not determine satisfiability within the given timeout.")

    # Call solve function to check for any solution
    is_sat, model, solution = solve(m, n, l, s, D, path, timeout)

    if is_sat:
        print("Found a solution:")
        for c in range(m):
            assigned_items = []
            for i in range(n+1):
                for j in range(n+2):
                    if is_true(model[path[c][i][j]]):
                        assigned_items.append(i)
                        break
            print(f"Courier {c}: [{', '.join(map(str, assigned_items))}]")  # Depict path starting and ending at depot

        # Print distances
        for c in range(m):
            distance = 0
            for i in range(n+1):
                for j in range(n+1):
                    if is_true(model[path[c][i][j]]):
                        distance += D[i][j]  # Adjust indices for D
            print(f"Distance for Courier {c}: {distance}")

    else:
        print("No solution found.")

    return solution


In [154]:
NUM_INST = 5
MCP(instances[NUM_INST-1])

Z3Exception: model is not available

In [134]:
# Function to check if the difference is positive (i.e., A > B)
def is_positive_difference(A: list[BoolRef], B: list[BoolRef]) -> BoolRef:
    # Calculate the difference
    diff_bits = calculate_distance(A, B)
    
    # The difference is positive if the MSB of the difference is 1
    return diff_bits[0]  # Return the most significant bit of the difference

In [135]:
# Function to perform full adder logic
def full_adder(A: BoolRef, B: BoolRef, Cin: BoolRef) -> tuple[BoolRef, BoolRef]:
    Sum = Xor(Xor(A, B), Cin)  # Calculate sum
    C_Out = Or(And(A, B), And(Or(A, B), Cin))  # Calculate carry-out
    return Sum, C_Out

In [136]:
# Example usage:
def calculate_distance_bin(max_dist, upperbound):
    # Convert max_dist and upperbound to binary
    max_dist_bin = bin(max_dist)[2:]  # Binary representation without '0b' prefix
    upperbound_bin = bin(upperbound)[2:]  # Binary representation without '0b' prefix
    
    # Perform binary addition using full adder
    sum_bits = []
    carry = 0
    
    # Ensure max_dist_bin and upperbound_bin are of the same length by padding with zeros if necessary
    max_len = max(len(max_dist_bin), len(upperbound_bin))
    max_dist_bin = max_dist_bin.zfill(max_len)
    upperbound_bin = upperbound_bin.zfill(max_len)
    
    for i in range(max_len):
        A = int(max_dist_bin[max_len - 1 - i])
        B = int(upperbound_bin[max_len - 1 - i])
        
        # Full adder calculation
        S, carry = full_adder(A, B, carry)
        sum_bits.append(S)
    
    # Append final carry-out if needed
    if carry:
        sum_bits.append(carry)
    
    # Convert sum_bits list back to a binary string
    sum_bits.reverse()  # Reverse to get the correct binary order
    sum_bin = ''.join(map(str, sum_bits))
    
    return sum_bin

In [137]:
import uuid

# Function to convert an integer to a list of BoolRefs of a specified length
def convert(number: int, length: int) -> list[BoolRef]:
    num = []
    str_num = bin(number)[2:].zfill(length)  # Binary representation as string, zero-padded to specified length
    for i in range(length):
        ni = Bool(f"bit_{i}_{str(uuid.uuid4())[:8]}")  # Create BoolRef with a unique name
        if str_num[i] == '1':
            num.append(ni)
        else:
            num.append(Not(ni))
    return num

In [138]:
# Function to calculate distance between two lists of BoolRefs representing binary numbers
def calculate_distance(max_dist: list[BoolRef], upperbound: list[BoolRef]) -> list[BoolRef]:
    sum_bits = []
    carry = BoolVal(False)  # Initial carry as BoolRef
    
    max_len = max(len(max_dist), len(upperbound))
    
    # Ensure max_dist and upperbound are of the same length by padding with False if necessary
    max_dist = max_dist + [BoolVal(False)] * (max_len - len(max_dist))
    upperbound = upperbound + [BoolVal(False)] * (max_len - len(upperbound))
    
    for i in range(max_len):
        A = max_dist[max_len - 1 - i]
        B = upperbound[max_len - 1 - i]
        
        # Full adder calculation
        S, carry = full_adder(A, B, carry)
        sum_bits.append(S)
    
    # Append final carry-out if needed
    if is_true(carry):
        sum_bits.append(BoolVal(True))  # Append True as BoolRef for carry-out
    
    sum_bits.reverse()  # Reverse to get the correct order
    
    return sum_bits

In [139]:
max_dist_int = 2
upperbound_int = 100

# Convert integers to lists of BoolRefs
max_dist = convert(max_dist_int, 4)  # Convert max_dist_int to 4-bit BoolRefs
upperbound = convert(upperbound_int, 7)  # Convert upperbound_int to 7-bit BoolRefs

# Calculate the sum using full adder
sum_bits = calculate_distance(max_dist, upperbound)

# Print the resulting sum_bits (list of BoolRefs)
print("Sum (BoolRefs):", sum_bits)

Sum (BoolRefs): [Xor(Xor(Not(bit_0_2c375faa), bit_0_af234da0),
    Or(And(Not(bit_1_d3be17fe), bit_1_6e37ea5a),
       And(Or(Not(bit_1_d3be17fe), bit_1_6e37ea5a),
           Or(And(bit_2_2cbe264f, Not(bit_2_a3a284b3)),
              And(Or(bit_2_2cbe264f, Not(bit_2_a3a284b3)),
                  Or(And(Not(bit_3_064c5c76),
                         Not(bit_3_160789cd)),
                     And(Or(Not(bit_3_064c5c76),
                            Not(bit_3_160789cd)),
                         Or(And(False, bit_4_4169056c),
                            And(Or(False, bit_4_4169056c),
                                Or(And(False,
                                       Not(bit_5_5f9c9b03)),
                                   And(Or(False,
                                        Not(bit_5_5f9c9b03)),
                                       Or(And(False,
                                        Not(bit_6_9e84cb78)),
                                        And(Or(False,
                           

In [140]:
def MCP(instance, timeout=5000):
    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']
    
    # Path variables: c courier went from node j to k
    path = [[[Bool(f"p_{c}_{i}_{j}") for j in range(n+2)] for i in range(n+2)] for c in range(m)]

    def solve(instance, m, n, l, s, D, path, timeout):
        solver = Solver()
        solver.set(timeout=timeout)  # Set timeout

        # All couriers start from depot (node n+1) and end at depot
        for c in range(m):
            solver.add(exactly_one_seq([path[c][n+1][j] for j in range(0, n+1)], f"start_{c}"))  # Start from node n+1
            solver.add(exactly_one_seq([path[c][i][n+1] for i in range(0, n+1)], f"end_{c}"))    # End at node n+1

        '''
        # Each item is taken by at most one courier
        for i in range(1, n+1):
            for j in range(1, n+2):
                solver.add(at_most_one_seq([path[c][i][j] for c in range(m)], f"item_{i}_{j}"))
        '''
        
        # Ensure all items are picked up
        for i in range(0, n+1):
            solver.add(exactly_one_seq([path[c][i][j] for c in range(m) for j in range(0, n+2)], f"all_items_{i}"))
        
        # Ensure each courier picks at least one item
        for c in range(m):
            solver.add(at_least_one_seq([path[c][i][j] for i in range(0, n+1) for j in range(0, n+1)]))

        # constraints based on distances
        distances = [[] for _ in range(m)]
        for c in range(m):
            for i in range(0, n+2):
                for j in range(0, n+2):
                    distances[c].extend([path[c][i][j]] * D[i-1][j-1])  # Adjust indices for D

        # Check for satisfiability
        if solver.check() == sat:
            model = solver.model()
            return True, model, [(c, i, j) for c in range(m) for i in range(1, n+2) for j in range(1, n+2) if is_true(model[path[c][i][j]])]
        else:
            return False, None, []

    lowerbound = 0
    best_solution = None
    best_distance = upperbound

    while lowerbound < upperbound:
        mid = (lowerbound + upperbound) // 2
        is_sat, solution, distance = solve(mid, m, n, l, s, D, path)
        if is_sat:
            best_solution = solution
            best_distance = distance
            upperbound = mid
        else:
            lowerbound = mid + 1

    return best_solution, best_distance

In [141]:
def MCP2(instance, timeout=5000):
    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']

    # Define maximum possible distance
    max_d = max([max(row) for row in D])
    upperbound = ceil(n/m) * max_d

    # Variables for paths
    path = [[[Bool(f"p_{c}_{i}_{j}") for j in range(n+1)] for i in range(n+1)] for c in range(m)]

    def solve_with_max_distance(upperbound, m, n, l, s, D, path):
        solver = Solver()
        solver.set(timeout=timeout)

        # all start from depot (node n) and end at depot
        for c in range(m):
            solver.add(exactly_one_seq([path[c][n][j] for j in range(n)], f"start_{c}"))
            solver.add(exactly_one_seq([path[c][i][n] for i in range(n)], f"end_{c}"))

        # each item is taken by at most one courier
        for i in range(n):
            item_constraints = []
            for c in range(m):
                item_constraints.extend([path[c][i][j] for j in range(n+1)])
            solver.add(at_most_one_seq(item_constraints, f"item_{i}"))

        # chain of couriers
        for c in range(m):
            for i in range(n+1):
                for j in range(n+1):
                    if i != j:
                        solver.add(Implies(path[c][i][j], exactly_one_seq([path[c][j][k] for k in range(n+1) if k != j], f"chain_{c}_{i}_{j}")))

        for c in range(m):
            for j in range(n+1):
                solver.add(at_most_one_seq([path[c][i][j] for i in range(n+1) if i != j], f"row_{c}_{j}"))

        for c in range(m):
            for i in range(n+1):
                solver.add(at_most_one_seq([path[c][i][j] for j in range(n+1) if i != j], f"col_{c}_{i}"))

        # Maximum load constraint
        for c in range(m):
            weight_set = []
            for i in range(n):
                weight_set.extend([path[c][i][j] for j in range(n+1)] * s[i])
            solver.add(at_most_k_seq(weight_set, l[c], f"load_{c}"))

        # Distance constraint
        distances = [[] for _ in range(m)]
        for c in range(m):
            for i in range(n+1):
                for j in range(n+1):
                    if i != j:
                        for k in range(D[i][j]):
                            distances[c].append(path[c][i][j])

        max_distance = Int('max_distance')
        for c in range(m):
            solver.add(And([Or(Not(distances[c][k]), max_distance > k) for k in range(len(distances[c]))]))

        solver.add(max_distance <= upperbound)

        if solver.check() == sat:
            model = solver.model()
            solution = [(c, i, j) for c in range(m) for i in range(n+1) for j in range(n+1) if model.evaluate(path[c][i][j])]
            return True, solution, model[max_distance].as_long()
        else:
            return False, [], 0

    lowerbound = 0
    best_solution = None
    best_distance = upperbound

    while lowerbound < upperbound:
        mid = (lowerbound + upperbound) // 2
        is_sat, solution, distance = solve_with_max_distance(mid, m, n, l, s, D, path)
        if is_sat:
            best_solution = solution
            best_distance = distance
            upperbound = mid
        else:
            lowerbound = mid + 1

    return best_solution, best_distance

## Testing

In [83]:
NUM_INST = 1
MCP(instances[NUM_INST-1])

Found a solution:
Courier 0: [0, 1, 2, 4, 6]
Courier 1: [3, 5]
Distance for Courier 0: 49
Distance for Courier 1: 4


[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 4),
 (0, 1, 5),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 4, 4),
 (0, 4, 7),
 (0, 6, 2),
 (0, 6, 3),
 (0, 6, 5),
 (0, 6, 6),
 (0, 7, 6),
 (1, 3, 0),
 (1, 5, 7),
 (1, 7, 6)]